In [1]:
# import the necessary libraries
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from collections import defaultdict

# setup 
#sns.set_style(style="whitegrid")
sns.set_style('ticks')

# display charts inline
%matplotlib inline

# to display all columns
pd.set_option('display.max_columns', None)


In [44]:
# to get column names
df = pd.read_csv('data/train_v2.csv', nrows=100)

In [46]:
# run above cell for nrows=100 to get following info
select_cols = list(set(df.columns) - set(['hits','customDimensions']))
select_cols

['socialEngagementType',
 'visitNumber',
 'channelGrouping',
 'visitStartTime',
 'fullVisitorId',
 'device',
 'visitId',
 'totals',
 'trafficSource',
 'geoNetwork',
 'date']

# Read train csv with json converters

In [52]:
# json_normalize doesn't work in google cloud/colab env. for all json columns. 
# df[column].apply(pd.Series) could be used instead.

def load_df(csv_path='data/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, usecols=select_cols,
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)
    print(df.columns)
    for column in JSON_COLUMNS:
        if(column == 'trafficSource'):
            column_as_df = json_normalize(df[column])
        else:
            column_as_df = df[column].apply(pd.Series)
        column_as_df.columns = ["{}.{}".format(column, subcolumn) for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print("Loaded {}. Shape: {}".format(os.path.basename(csv_path), df.shape))
    return df

In [ ]:
%%time
# load the data, test data is only for the submission to kaggle
train_df = load_df()

#  load at the time of final reporting
#test_df = load_df("test.csv")

In [2]:
train_df = pd.read_pickle('data/test_v2_clean.pkl')

In [3]:
train_df.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions_value
0,Organic Search,20180511,7460955084541987166,1526099341,2,1526099341,Chrome,mobile,True,Android,(not set),Asia,India,(not set),unknown.unknown,Delhi,Southern Asia,NaN,4,NaN,3,1,973,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google,APAC
1,Direct,20180511,460252456180441002,1526064483,166,1526064483,Chrome,desktop,False,Macintosh,San Francisco,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,4,NaN,3,1,49,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct),North America
2,Organic Search,20180511,3461808543879602873,1526067157,2,1526067157,Chrome,desktop,False,Chrome OS,not available in demo dataset,Americas,United States,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,Northern America,NaN,4,NaN,3,1,24,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google,North America
3,Direct,20180511,975129477712150630,1526107551,4,1526107551,Chrome,mobile,True,iOS,Houston,Americas,United States,Houston TX,(not set),Texas,Northern America,NaN,5,NaN,4,1,25,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct),North America
4,Organic Search,20180511,8381672768065729990,1526060254,1,1526060254,Internet Explorer,tablet,True,Windows,Irvine,Americas,United States,Los Angeles CA,com,California,Northern America,NaN,5,1,4,1,49,NaN,NaN,NaN,(not set),NaN,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,(not set),google,North America


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 39 columns):
channelGrouping                                 401589 non-null object
date                                            401589 non-null int64
fullVisitorId                                   401589 non-null object
visitId                                         401589 non-null int64
visitNumber                                     401589 non-null int64
visitStartTime                                  401589 non-null int64
device_browser                                  401589 non-null object
device_deviceCategory                           401589 non-null object
device_isMobile                                 401589 non-null bool
device_operatingSystem                          401589 non-null object
geoNetwork_city                                 401589 non-null object
geoNetwork_continent                            401589 non-null object
geoNetwork_country                         

# Metadata for columns

In [5]:
# metadata - number of unique values and null values for each column
info = pd.DataFrame(index=train_df.columns.values,data={'data type': train_df.dtypes, \
                          'unique values': train_df.nunique(), 'null values': train_df.isnull().sum()})

In [6]:
info.sort_values(['unique values','null values'])

,data type,null values,unique values
totals_newVisits,object,115524,1
totals_bounces,object,218911,1
trafficSource_isTrueDirect,object,253180,1
trafficSource_adwordsClickInfo.isVideoAd,object,390984,1
device_isMobile,bool,0,2
trafficSource_adwordsClickInfo.adNetworkType,object,390984,2
trafficSource_adwordsClickInfo.page,object,390984,2
device_deviceCategory,object,0,3
customDimensions_value,object,60581,5
trafficSource_adwordsClickInfo.slot,object,390984,5


# Convert one valued columns to boolean

In [7]:
# single valued columns but having nulls
for col in info[(info['unique values']==1) & (info['null values'] > 0)].index:
    print(col, train_df[col].unique())

totals_bounces [nan '1']
totals_newVisits [nan '1']
trafficSource_adwordsClickInfo.isVideoAd [nan False]
trafficSource_isTrueDirect [True nan]


In [8]:
# change boolean variables into 1/0 number
def boolean_to_one_zero(col, value, bool_value):
    if bool_value:
        train_df.loc[train_df[col] == value, col] = 1
        train_df[col].fillna(0,inplace=True)
    else:
        train_df.loc[train_df[col] == value, col] = 0
        train_df[col].fillna(1,inplace=True)
    train_df[col] = train_df[col].astype(int)

In [9]:
# convert nans to either 0 or 1
boolean_to_one_zero('totals_bounces',1,True)
boolean_to_one_zero('totals_newVisits',1,True)
boolean_to_one_zero('trafficSource_adwordsClickInfo.isVideoAd',False,False)
#boolean_to_one_zero('trafficSource.campaignCode','11251kjhkvahf',False)
boolean_to_one_zero('trafficSource_isTrueDirect',True,True)

In [10]:
# verify changed columns
for col in info[(info['unique values']==1) & (info['null values'] > 0)].index:
    print(col, len(train_df[col].unique()))

totals_bounces 2
totals_newVisits 2
trafficSource_adwordsClickInfo.isVideoAd 2
trafficSource_isTrueDirect 2


# Delete non useful columns

In [11]:
# Columns with 1 unique value and zero null value might not be useful
# let us check those columns

def single_valued_columns(df, meta_info):
    single_valued_columns = []
    for col in meta_info[(meta_info['unique values']==1) & (meta_info['null values'] ==0)].index:
        unique_value = df[col].unique()
        print(col, unique_value)
        single_valued_columns.append(col)
    return single_valued_columns

In [12]:
# these columns can be deleted - columns names ['unique-value']
cols_to_delete = single_valued_columns(train_df, info)

In [13]:
cols_to_delete_df = pd.DataFrame(cols_to_delete, columns=['cols_to_delete'])

In [14]:
cols_to_delete_df.to_csv("data/cols_to_delete.csv")

In [15]:
# drop the columns above
for col in cols_to_delete:
    train_df.drop(col, inplace=True, axis=1)

In [16]:
# what's the new shape of train data after deleting columns
train_df.shape

(401589, 39)

# Columns having more than one value other than nulls 
**pageviews, transactionRevenue, adContent, adNetworkType, gclId, page, slot, keyword, referralPath**

In [17]:
# multi valued columns and having nulls
# categorical columns here would have to be one hot encoded later on or binned in some way
for col in info[(info['unique values']>1) & (info['null values'] > 0)].index:
    print(col, train_df[col].nunique(), train_df[col].isnull().sum())

totals_pageviews 155 101
totals_timeOnSite 3578 183686
totals_totalTransactionRevenue 2461 396995
totals_transactionRevenue 1950 396995
totals_transactions 7 395284
trafficSource_adwordsClickInfo.adNetworkType 2 390984
trafficSource_adwordsClickInfo.gclId 9016 390977
trafficSource_adwordsClickInfo.page 2 390984
trafficSource_adwordsClickInfo.slot 5 390984
trafficSource_keyword 677 40226
customDimensions_value 5 60581


In [18]:
train_df['totals_hits'].unique()

array(['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
       '39', '40', '41', '42', '43', '47', '51', '52', '53', '54', '56',
       '59', '61', '62', '65', '67', '74', '87', '2', '3', '1', '38',
       '44', '45', '49', '50', '55', '57', '66', '70', '71', '77', '81',
       '82', '89', '94', '48', '58', '60', '64', '68', '83', '102', '46',
       '69', '88', '95', '116', '127', '178', '73', '159', '79', '107',
       '135', '110', '100', '103', '104', '111', '500', '174', '63',
       '109', '189', '90', '140', '222', '72', '188', '211', '128', '78',
       '98', '137', '101', '112', '96', '99', '108', '130', '139', '76',
       '91', '92', '187', '152', '132', '158', '301', '106', '117', '181',
       '120', '93', '141', '172', '75', '85', '126', '246', '84', '113',
       '118', '269', '119', '234', '138', '105', '80', 

In [19]:
def fill_nans(test_df):
    # fillna and make it int
    test_df['totals_pageviews'].fillna(0, inplace=True)
    test_df['totals_pageviews'] = test_df['totals_pageviews'].astype(int)

    # hits to int
    test_df['totals_hits'] = test_df['totals_hits'].astype(int)

    # this is critical filling up 0 for nan now, later there can be a separate boolean column for 0 and non-zero
    # trafficSource.adwordsClickInfo.page
    # fillna and make it int
    test_df['trafficSource_adwordsClickInfo.page'].fillna(0, inplace=True)
    test_df['trafficSource_adwordsClickInfo.page'] = test_df['trafficSource_adwordsClickInfo.page'].astype(int)

    # trafficSource.adwordsClickInfo.slot, trafficSource.keyword,trafficSource.referralPath
    more_cols = ['trafficSource_adwordsClickInfo.adNetworkType','trafficSource_adwordsClickInfo.gclId',\
                 'trafficSource_adwordsClickInfo.slot','trafficSource_adContent',\
                'trafficSource_keyword','trafficSource_referralPath']
    for col in more_cols:
        print(test_df[col].unique()[0:10])
        test_df[col].fillna('Unknown', inplace=True)
    return test_df

In [20]:
# actually this type of mutable function call is not good practice
fill_nans(train_df)

[nan 'Google Search' 'Content']
[nan
 'CjwKCAiAz-7UBRBAEiwAVrz-9XDp4Vu2734OvwUvrbhvEyQcbsYZKVHR_h5uJJxt9xEZ1y0rU2zuThoC2BYQAvD_BwE'
 'EAIaIQobChMI1N2pgJXl2gIViGnBCh0lvgJ1EAEYASAAEgLQdfD_BwE'
 'EAIaIQobChMIg7ea6KLl2gIVCZXICh0VTgpLEAEYASAAEgJQIvD_BwE'
 'EAIaIQobChMI4pzYjs2h2wIVk2IBCh3gzwd-EAEYAiAAEgJU9PD_BwE'
 'CjwKCAjwi6TYBRAYEiwAOeH7GQ222lZGJZLfR_8WAAgw0MFbPjh_QY54omdVaU_cJ1NQ1eElsdUs0RoC3T0QAvD_BwE'
 'EAIaIQobChMIzZj1hMKg2wIVTL13Ch3FKws_EAEYASAAEgL23_D_BwE'
 'EAIaIQobChMIk97B7rT63AIVlcBkCh2PTANzEAAYASAAEgJcV_D_BwE'
 'Cj0KCQjwquTbBRCSARIsADzW88xA4SIzIwHZM83TPCP-1wLAifST1Bm7sJL7hVW6BuJNOAAq9rYMi4caAluBEALw_wcB'
 'Cj0KCQjwquTbBRCSARIsADzW88zpRLhwN7ZWOWyNhGCo3pyrI1ELU_T61X_ElSWUHkLUbxGbsMAM8loaAnadEALw_wcB']
[nan 'Top' 'Google Display Network' 'Google search: Top' 'RHS'
 'Google search: Other']
['(not set)' 'Google Merchandise Collection'
 'Smart display ad - 8/17/2017' 'BQ' 'YouTube Merchandise Collection'
 'Official Google Merchandise' 'YouTube Apparel Merchandise'
 'Google Office Merch

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions_value
0,Organic Search,20180511,7460955084541987166,1526099341,2,1526099341,Chrome,mobile,True,Android,(not set),Asia,India,(not set),unknown.unknown,Delhi,Southern Asia,0,4,0,3,1,973,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,APAC
1,Direct,20180511,460252456180441002,1526064483,166,1526064483,Chrome,desktop,False,Macintosh,San Francisco,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,4,0,3,1,49,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not set),(none),(not set),(direct),North America
2,Organic Search,20180511,3461808543879602873,1526067157,2,1526067157,Chrome,desktop,False,Chrome OS,not available in demo dataset,Americas,United States,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,Northern America,0,4,0,3,1,24,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,North America
3,Direct,20180511,975129477712150630,1526107551,4,1526107551,Chrome,mobile,True,iOS,Houston,Americas,United States,Houston TX,(not set),Texas,Northern America,0,5,0,4,1,25,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not set),(none),(not set),(direct),North America
4,Organic Search,20180511,8381672768065729990,1526060254,1,1526060254,Internet Explorer,tablet,True,Windows,Irvine,Americas,United States,Los Angeles CA,com,California,Northern America,0,5,1,4,1,49,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,(not set),google,North America
5,Organic Search,20180511,2866297766347322467,1526061951,2,1526061951,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Brazil,not available in demo dataset,vivozap.com.br,not available in demo dataset,South America,0,5,0,4,1,120,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,South America
6,Direct,20180511,2235365487897339889,1526062356,1,1526062356,Chrome,mobile,True,Android,San Jose,Americas,United States,San Francisco-Oakland-San Jose CA,tmodns.net,California,Northern America,0,5,1,4,1,58,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not set),(none),(not set),(direct),North America
7,Organic Search,20180511,1303090465617023038,1526078660,3,1526078660,Chrome,desktop,False,Macintosh,Mountain View,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,5,0,4,1,56,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,North America
8,Affiliates,20180511,0459669224143241747,1526054953,1,1526054953,Safari (in-app),tablet,True,iOS,not available in demo dataset,Asia,Taiwan,not available in demo dataset,hinet.net,not available in demo dataset,Eastern Asia,0,5,1,3,1,57,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,Data Share Promo,0,(not set),affiliate,(not set),Partners,APAC
9,Direct,20180511,589440789980446309,1526051458,1,1526051458,Edge,desktop,False,Windows,Madrid,Europe,Spain,(not set),urjc.es,Madrid,Southern Europe,0,5,1,3,1,22,NaN,NaN,NaN,(not set)

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 39 columns):
channelGrouping                                 401589 non-null object
date                                            401589 non-null int64
fullVisitorId                                   401589 non-null object
visitId                                         401589 non-null int64
visitNumber                                     401589 non-null int64
visitStartTime                                  401589 non-null int64
device_browser                                  401589 non-null object
device_deviceCategory                           401589 non-null object
device_isMobile                                 401589 non-null bool
device_operatingSystem                          401589 non-null object
geoNetwork_city                                 401589 non-null object
geoNetwork_continent                            401589 non-null object
geoNetwork_country                         

# Save data to disk

In [22]:
#train_df.to_csv('data/cleaned_train.csv')
train_df.to_csv('data/cleaned_test.csv')

# Check validity of Geo columns

In [23]:
# all geo columns
geo_cols = ['geoNetwork_city','geoNetwork_metro','geoNetwork_region','geoNetwork_country', \
            'geoNetwork_subContinent','geoNetwork_continent']

In [24]:
# how many unique geo combinations
geo_df = train_df[geo_cols].drop_duplicates()
geo_df

,geoNetwork_city,geoNetwork_metro,geoNetwork_region,geoNetwork_country,geoNetwork_subContinent,geoNetwork_continent
0,(not set),(not set),Delhi,India,Southern Asia,Asia
1,San Francisco,San Francisco-Oakland-San Jose CA,California,United States,Northern America,Americas
2,not available in demo dataset,not available in demo dataset,not available in demo dataset,United States,Northern America,Americas
3,Houston,Houston TX,Texas,United States,Northern America,Americas
4,Irvine,Los Angeles CA,California,United States,Northern America,Americas
5,not available in demo dataset,not available in demo dataset,not available in demo dataset,Brazil,South America,Americas
6,San Jose,San Francisco-Oakland-San Jose CA,California,United States,Northern America,Americas
7,Mountain View,San Francisco-Oakland-San Jose CA,California,United States,Northern America,Americas
8,not available in demo dataset,not available in demo dataset,not available in demo dataset,Taiwan,Eastern Asia,Asia
9,Madrid,(not set),Madrid,Spain,Southern Europe,Europe


In [25]:
# what are the number of rows in data set for each unique combination
unique_geo_counts = train_df.groupby(geo_cols).count()

In [26]:
unique_geo_counts = unique_geo_counts.iloc[:,0:1].reset_index().sort_values('channelGrouping').rename(columns={'channelGrouping':'counts'})
unique_geo_counts

,geoNetwork_city,geoNetwork_metro,geoNetwork_region,geoNetwork_country,geoNetwork_subContinent,geoNetwork_continent,counts
787,Redwood City,San Francisco-Oakland-San Jose CA,California,Ireland,Northern Europe,Europe,1
339,Cupertino,San Francisco-Oakland-San Jose CA,California,Canada,Northern America,Americas,1
639,Mountain View,San Francisco-Oakland-San Jose CA,California,Australia,Australasia,Oceania,1
986,Tel Aviv-Yafo,(not set),Tel Aviv District,Russia,Eastern Europe,Europe,1
987,Tel Aviv-Yafo,(not set),Tel Aviv District,United Kingdom,Northern Europe,Europe,1
643,Mountain View,San Francisco-Oakland-San Jose CA,California,Colombia,South America,Americas,1
988,Tel Aviv-Yafo,(not set),Tel Aviv District,United States,Northern America,Americas,1
644,Mountain View,San Francisco-Oakland-San Jose CA,California,France,Western Europe,Europe,1
852,San Jose,San Francisco-Oakland-San Jose CA,California,Argentina,South America,Americas,1
645,Mountain View,San Francisco-Oakland-San Jose CA,California,Guatemala,Central America,Americas,1


In [27]:
unique_city_counts = geo_df.groupby(['geoNetwork_city']).size().reset_index(name='unique_city_counts')

In [28]:
unique_city_counts = unique_city_counts[(unique_city_counts['geoNetwork_city'] != '(not set)') & \
                                       (unique_city_counts['geoNetwork_city'] != 'not available in demo dataset') &
                                       (unique_city_counts['unique_city_counts'] > 1)]

In [29]:
unique_city_counts

,geoNetwork_city,unique_city_counts
11,Ahmedabad,2
21,Amsterdam,7
23,Ann Arbor,4
31,Atlanta,3
32,Auckland,3
33,Austin,6
37,Bandung,2
38,Bangkok,3
39,Barcelona,3
41,Barrie,2


In [30]:
invalid_geo_data_df = pd.DataFrame(columns=geo_df.columns)

for city in unique_city_counts['geoNetwork_city']:
    invalid_geo_data_df = invalid_geo_data_df.append(unique_geo_counts[unique_geo_counts['geoNetwork_city']==city].sort_values('counts',ascending=False)[1:])

# there are the wrong geo info rows
invalid_geo_data_df

,counts,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_region,geoNetwork_subContinent
177,1.0,Ahmedabad,Asia,Nepal,(not set),Gujarat,Southern Asia
189,6.0,Amsterdam,Europe,Germany,(not set),North Holland,Western Europe
193,5.0,Amsterdam,Americas,United States,(not set),North Holland,Northern America
192,2.0,Amsterdam,Asia,Singapore,(not set),North Holland,Southeast Asia
188,2.0,Amsterdam,Europe,France,(not set),North Holland,Western Europe
187,2.0,Amsterdam,Europe,Belgium,(not set),North Holland,Western Europe
190,1.0,Amsterdam,Europe,Ireland,(not set),North Holland,Northern Europe
197,2.0,Ann Arbor,Americas,Canada,Detroit MI,Michigan,Northern America
195,1.0,Ann Arbor,Oceania,Australia,Detroit MI,Michigan,Australasia
196,1.0,Ann Arbor,Americas,Brazil,Detroit MI,Michigan,South America


In [31]:
invalid_geo_data_df['counts'].sum()

1891.0

In [32]:
def get_indices(row):
    return train_df[(train_df['geoNetwork_city']==row[1][0]) & \
             (train_df['geoNetwork_metro']==row[1][1]) & \
             (train_df['geoNetwork_region']==row[1][2]) & \
             (train_df['geoNetwork_country']==row[1][3])
            ].index

In [33]:
%%time
# invalid geo combo data
delete_row_indices = []

for row in invalid_geo_data_df.iterrows():
    delete_row_indices.extend(list(get_indices(row)))

CPU times: user 44.2 s, sys: 18.8 ms, total: 44.2 s
Wall time: 44.2 s


In [34]:
len(delete_row_indices)

0

In [35]:
# delete invalid geo combo data
train_df.drop(delete_row_indices,inplace=True,axis=0)

In [36]:
train_df.shape

(401589, 39)

# Save data to disk

In [37]:
#train_df.to_csv('data/cleaned_train.csv')
train_df.to_csv('data/cleaned_test.csv')

# Extract date time fields

In [38]:
def get_time_fields(x):
    utc = pd.datetime.utcfromtimestamp(x)
    result = {'year': utc.year, 'month' : utc.month,'day': utc.day, 'weekday': utc.weekday(), 'hour': utc.hour}
    return pd.Series(result)

In [39]:
time_fields = train_df['visitStartTime'].apply(get_time_fields)
time_fields

,day,hour,month,weekday,year
0,12,4,5,5,2018
1,11,18,5,4,2018
2,11,19,5,4,2018
3,12,6,5,5,2018
4,11,17,5,4,2018
5,11,18,5,4,2018
6,11,18,5,4,2018
7,11,22,5,4,2018
8,11,16,5,4,2018
9,11,15,5,4,2018


In [40]:
train_df = train_df.join(time_fields)
train_df

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions_value,day,hour,month,weekday,year
0,Organic Search,20180511,7460955084541987166,1526099341,2,1526099341,Chrome,mobile,True,Android,(not set),Asia,India,(not set),unknown.unknown,Delhi,Southern Asia,0,4,0,3,1,973,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,APAC,12,4,5,5,2018
1,Direct,20180511,460252456180441002,1526064483,166,1526064483,Chrome,desktop,False,Macintosh,San Francisco,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,4,0,3,1,49,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not set),(none),(not set),(direct),North America,11,18,5,4,2018
2,Organic Search,20180511,3461808543879602873,1526067157,2,1526067157,Chrome,desktop,False,Chrome OS,not available in demo dataset,Americas,United States,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,Northern America,0,4,0,3,1,24,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,North America,11,19,5,4,2018
3,Direct,20180511,975129477712150630,1526107551,4,1526107551,Chrome,mobile,True,iOS,Houston,Americas,United States,Houston TX,(not set),Texas,Northern America,0,5,0,4,1,25,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not set),(none),(not set),(direct),North America,12,6,5,5,2018
4,Organic Search,20180511,8381672768065729990,1526060254,1,1526060254,Internet Explorer,tablet,True,Windows,Irvine,Americas,United States,Los Angeles CA,com,California,Northern America,0,5,1,4,1,49,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),0,(not provided),organic,(not set),google,North America,11,17,5,4,2018
5,Organic Search,20180511,2866297766347322467,1526061951,2,1526061951,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Brazil,not available in demo dataset,vivozap.com.br,not available in demo dataset,South America,0,5,0,4,1,120,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,South America,11,18,5,4,2018
6,Direct,20180511,2235365487897339889,1526062356,1,1526062356,Chrome,mobile,True,Android,San Jose,Americas,United States,San Francisco-Oakland-San Jose CA,tmodns.net,California,Northern America,0,5,1,4,1,58,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not set),(none),(not set),(direct),North America,11,18,5,4,2018
7,Organic Search,20180511,1303090465617023038,1526078660,3,1526078660,Chrome,desktop,False,Macintosh,Mountain View,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,0,5,0,4,1,56,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,(not set),1,(not provided),organic,(not set),google,North America,11,22,5,4,2018
8,Affiliates,20180511,0459669224143241747,1526054953,1,1526054953,Safari (in-app),tablet,True,iOS,not available in demo dataset,Asia,Taiwan,not available in demo dataset,hinet.net,not available in demo dataset,Eastern Asia,0,5,1,3,1,57,NaN,NaN,NaN,(not set),Unknown,Unknown,1,0,Unknown,Data Share Promo,0,(not set),affiliate,(not set),Partners,APAC,11,16,5,4,2018
9,Direct,20180511,5894

# Delete second set of columns

In [41]:
# columns to drop set 2
cols_to_drop = []

# id fields serve no purpose
#cols_to_drop.append('fullVisitorId')
#cols_to_drop.append('sessionId')
cols_to_drop.append('visitId')

# date is redundant because visitStartTime includes date
#cols_to_drop.append('date')

# visitStartTime is broken into day, month etc. and exact time upto seconds doesn't make business sense
cols_to_drop.append('visitStartTime')

# continent is redundant becuase sub-continent is bit more granular but includes continent info
cols_to_drop.append('geoNetwork_continent')

# isMobile is redundant, device category already have that category
cols_to_drop.append('device_isMobile')

train_df.drop(cols_to_drop, axis=1, inplace=True)

In [42]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401589 entries, 0 to 401588
Data columns (total 39 columns):
channelGrouping                                 401589 non-null object
fullVisitorId                                   401589 non-null object
visitNumber                                     401589 non-null int64
device_browser                                  401589 non-null object
device_deviceCategory                           401589 non-null object
device_operatingSystem                          401589 non-null object
geoNetwork_city                                 401589 non-null object
geoNetwork_country                              401589 non-null object
geoNetwork_metro                                401589 non-null object
geoNetwork_networkDomain                        401589 non-null object
geoNetwork_region                               401589 non-null object
geoNetwork_subContinent                         401589 non-null object
totals_bounces                        

# Save data to disk

In [43]:
#train_df.to_csv('data/cleaned_train.csv')
train_df.to_csv('data/cleaned_test.csv')

In [44]:
# need to add back date ...was deleted by mistake
date_df = pd.read_pickle('data/test_v2_clean.pkl')

In [45]:
date_df['date'].shape

(401589,)

In [49]:
pd.concat([train_df,date_df['date']],axis=1).to_csv('data/cleaned_test.csv')